In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Creating an Agent with Gemini's Function Calling

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/iamthuya/google-cloud-workshops/blob/main/ai-agents/gemini/creating_an_agent_with_function_calling.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fiamthuya%2Fgoogle-cloud-workshops%2Fmain%2Fai-agents%2Fgemini%2Fcreating_an_agent_with_function_calling.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/iamthuya/google-cloud-workshops/blob/main/ai-agents/gemini/creating_an_agent_with_function_calling.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/iamthuya/google-cloud-workshops/blob/main/ai-agents/gemini/creating_an_agent_with_function_calling.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>

| | |
|-|-|
|Author(s) | [Thu Ya Kyaw](https://github.com/iamthuya) |

### Install Vertex AI SDK for Python

Here you will install required packages for this lab.

In [ ]:
! pip install --upgrade --quiet --user google-cloud-aiplatform

### Restart current runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

Don't worry if you see a notification message like `Your session crashed for an unknown reason.` This is expected as you are shutting down the kernel from the same instance using code.


In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>

### Authenticate your environment (Google Colab only)

If you are running this notebook on Google Colab, uncomment and run the following cell to authenticate your environment. You **don't need to authenticate** if you are using Colab Enterprise.

In [ ]:
# from google.colab import auth

# auth.authenticate_user()

### Initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [1]:
import vertexai

PROJECT_ID = "your-project-id"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

assert PROJECT_ID != "your-project-id", "Please input your project id"
assert LOCATION != "your-location", "Please input your project location"

vertexai.init(project=PROJECT_ID, location=LOCATION)

In the next cell, you will import the required modules from install packages.

In [2]:
import requests

from vertexai.generative_models import (
    GenerativeModel,
    FunctionDeclaration,
    Tool,
    Part,

)

### Creating an Agent with Gemini

In the following cell, you will create an agent, start a chat session, and ask about current exchange rate from USD to SGD.

In [3]:
# create an agent
agent = GenerativeModel(
    model_name="gemini-1.5-flash"
)

# start a chat session
chat = agent.start_chat()

# ask the agent about exchange rate
prompt = "what's the current exchange rate of USD to SGD?"
response = chat.send_message(prompt)

# display the response
response.candidates[0].content.parts[0]

text: "I do not have access to real-time information, including live exchange rates. \n\nTo get the most up-to-date USD to SGD exchange rate, I recommend checking a reputable online currency converter like:\n\n* **Google Finance:** Just search \"USD to SGD\" in Google.\n* **XE.com:** A popular website dedicated to exchange rates.\n* **Bloomberg:** A financial news platform with real-time data.\n\nThese websites will provide you with the current exchange rate, as well as historical data and charts. \n"

The model is smart enough to provide alternative ways to get the exchange rate instead of confidently response with a number (hallucination). However, it is generally better to provide the exchange rate right away to the user.

This is where function calling feature comes in. With function calling feature, you can create a tool that can retrieve latest exchange rate for the agent. You will do that in the following sections.

Firstly, let's create a function that can get the current exchange rate using the API.

In [4]:
# define a function to get exchange rate
def get_fx_rate(base_currency: str, target_currency: str):
    """
    Fetches the current exchange rate between two currencies.

    Args:
        base_currency: The base currency (e.g., "USD").
        target_currency: The target currency (e.g., "SGD").

    Returns:
        The exchange rate information as a json response,
        or None if the rate could not be fetched.
    """

    url = f"https://hexarate.paikama.co/api/rates/latest/{base_currency}?target={target_currency}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()


# test the function
get_fx_rate("USD", "SGD")

{'status_code': 200,
 'data': {'base': 'USD',
  'target': 'SGD',
  'mid': 1.34565,
  'unit': 1,
  'timestamp': '2024-11-22T02:01:34.799Z'}}

The function is working as intented. However, in order for Gemini's function calling feature to detect that Python function, you will need to create a `FunctionDeclaration` of that Python function. You will do that in the next cell.

In [5]:
# initialize function declaration
fd_get_fx_rate = FunctionDeclaration(
    name = "get_fx_rate",
    description = "Fetches the current exchange rate between two currencies.",
    parameters = {
        "type" : "object",
        "properties" : {
            "base_currency": {
                "type": "string",
                "description": "The base currency (e.g., 'USD')"
            },
            "target_currency": {
                "type": "string",
                "description": "The target currency (e.g., 'SGD')"
            },
        },
    }
)

Once the declaration is done, you need to create a `Tool` with the declaration so that you can attach it to the agent.

In [6]:
# create a tool with the function declaration
currency_tool = Tool(
    function_declarations=[
        fd_get_fx_rate
    ]
)

Now, you are ready to create an agent with a tool that has access to the function to retrive currency exchange rate.

In [7]:
# create an agent with the currency tool
agent = GenerativeModel(
    model_name="gemini-1.5-flash",
    tools=[currency_tool],
)

# start a chat session
chat = agent.start_chat()

# ask the agent about exchange rate
response = chat.send_message(prompt)

# display the response
response.candidates[0].content.parts[0]

function_call {
  name: "get_fx_rate"
  args {
    fields {
      key: "base_currency"
      value {
        string_value: "USD"
      }
    }
    fields {
      key: "target_currency"
      value {
        string_value: "SGD"
      }
    }
  }
}

You can see that the response is different from the previous instance. The agent is now returning a function call response instead of a text response.

The agent will now wait for you to call the function as stated in the function call response. Only when the agent recieves back a response from that function (stated in the function call), it will answer back to the user.

The information from function call response can be assesed directly using `.function_call` attribute.

In [8]:
response.candidates[0].content.parts[0].function_call

name: "get_fx_rate"
args {
  fields {
    key: "base_currency"
    value {
      string_value: "USD"
    }
  }
  fields {
    key: "target_currency"
    value {
      string_value: "SGD"
    }
  }
}

In an ideal scenario, you will be automating the whole process. Here you will try calling the function manually using the information provided from function call response of the agent.

In [9]:
# assign function name and arguments as provided in the function call response
func_name = response.candidates[0].content.parts[0].function_call.name
func_args = response.candidates[0].content.parts[0].function_call.args

# call the function to get the exchange rate information
fx_rate_json = globals()[func_name](**func_args)
fx_rate_json

{'status_code': 200,
 'data': {'base': 'USD',
  'target': 'SGD',
  'mid': 1.34565,
  'unit': 1,
  'timestamp': '2024-11-22T02:01:34.799Z'}}

Awesome! The function returns a JSON response with exchange rate information. Now, you need to pass that information back to the agent so that it can answer to the user.

In [10]:
# provide function's response to the chat instance
response = chat.send_message(
    Part.from_function_response(
        name=func_name,
        response={
            "content": fx_rate_json,
        },
    ),
)

response.text

'The current exchange rate for USD to SGD is 1.34565. \n'

# Conclusions

You have explored how to create an agent using Gemini's function calling feature through the Vertex AI Python SDK.